<a href="https://colab.research.google.com/github/asigalov61/SuperPiano/blob/master/Super_Piano_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Super Piano 3: Google Music Transformer
## Generating Music with Long-Term structure
### Based on 2019 ICLR paper by Cheng-Zhi Anna Huang, Google Brain and Damon Gwinn's code/repo https://github.com/gwinndr/MusicTransformer-Pytorch

Huge thanks go out to the following people who contributed the code/repos used in this colab. Additional contributors are listed in the code as well.

1) Kevin-Yang https://github.com/jason9693/midi-neural-processor

2) gudgud96 for fixing Kevin's MIDI Encoder properly https://github.com/gudgud96

2) jinyi12, Zac Koh, Akamight, Zhang https://github.com/COMP6248-Reproducability-Challenge/music-transformer-comp6248

Thank you so much for your hard work and for sharing it with the world :)


###Setup Environment and Dependencies. Check GPU.

In [47]:
#@title Import all needed modules
import numpy as np
import pickle
import os
import sys
import math
import random
# For plotting
import pypianoroll
from pypianoroll import Multitrack, Track
import matplotlib
import matplotlib.pyplot as plt
#matplotlib.use('SVG')
#%matplotlib inline
#matplotlib.get_backend()
# import mir_eval.display
import librosa
import librosa.display
# For rendering output audio
import pretty_midi
from midi2audio import FluidSynth
# from google.colab import output
from IPython.display import display, Javascript, HTML, Audio

In [48]:
# #@title Prepare directory sctructure and MIDI processor
# %cd /con/tent/
# !mv midi-neural-processor midi_processor
# %cd /content/MusicTransformer-Pytorch/

In [49]:
exp = [5, 2, 1]
#@title Process MAESTRO MIDI DataSet
!python3 preprocess_midi.py '../data_mt3/exp{exp[0]}-{exp[1]}/{exp[0]}-{exp[1]}-{exp[2]}'

Preprocessing midi files and saving to ./model_ref_MT
Found 1000 pieces
Preprocessing custom data...
50 / 1000
100 / 1000
150 / 1000
200 / 1000
250 / 1000
300 / 1000
350 / 1000
400 / 1000
450 / 1000
500 / 1000
550 / 1000
600 / 1000
650 / 1000
700 / 1000
750 / 1000
800 / 1000
850 / 1000
900 / 1000
950 / 1000
1000 / 1000
Done!



In [50]:
import pandas as pd

# อ่านไฟล์ CSV
df1 = pd.read_csv(f'../model_ref_MT/data_tokenize/tokenize{exp[0]}-{exp[1]}-{exp[2]}.csv')
df2 = pd.read_csv(f'../data/exp{exp[0]}-{exp[1]}/{exp[0]}-{exp[1]}-{exp[2]}/file_names_mt3_exp{exp[0]}-{exp[1]}-{exp[2]}_train.csv')
joined_df = pd.merge(df1, df2, on='File Name wav', how='inner')
joined_df = joined_df.rename(columns={
    'label_x': 'label',
    'label_y': 'label_mt3'
    # เพิ่มชื่อคอลัมน์ที่ต้องการเปลี่ยนเพิ่มเติมที่นี่
})
# เลือกคอลัมน์ที่ต้องการ
selected_columns = ['File Name wav','label','tempo','rhythm','rest','num of pitch','File Name midi']
result_df = joined_df[selected_columns]
# บันทึกเป็นไฟล์ CSV ใหม่
result_df.to_csv(f'../data/exp{exp[0]}-{exp[1]}/{exp[0]}-{exp[1]}-{exp[2]}/file_names_MT_exp{exp[0]}-{exp[1]}-{exp[2]}_train.csv', index=False)


# อ่านไฟล์ CSV

df2 = pd.read_csv(f'../data/exp{exp[0]}-{exp[1]}/{exp[0]}-{exp[1]}-{exp[2]}/file_names_mt3_exp{exp[0]}-{exp[1]}-{exp[2]}_test.csv')
joined_df = pd.merge(df1, df2, on='File Name wav', how='inner')
joined_df = joined_df.rename(columns={
    'label_x': 'label',
    'label_y': 'label_mt3'
    # เพิ่มชื่อคอลัมน์ที่ต้องการเปลี่ยนเพิ่มเติมที่นี่
})
# เลือกคอลัมน์ที่ต้องการ
selected_columns = ['File Name wav','label','tempo','rhythm','rest','num of pitch','File Name midi']
result_df = joined_df[selected_columns]
# บันทึกเป็นไฟล์ CSV ใหม่
result_df.to_csv(f'../data/exp{exp[0]}-{exp[1]}/{exp[0]}-{exp[1]}-{exp[2]}/file_names_MT_exp{exp[0]}-{exp[1]}-{exp[2]}_test.csv', index=False)


# อ่านไฟล์ CSV

df2 = pd.read_csv(f'../data/exp{exp[0]}-{exp[1]}/{exp[0]}-{exp[1]}-{exp[2]}/file_names_mt3_exp{exp[0]}-{exp[1]}-{exp[2]}_validate.csv')
joined_df = pd.merge(df1, df2, on='File Name wav', how='inner')
joined_df = joined_df.rename(columns={
    'label_x': 'label',
    'label_y': 'label_mt3'
    # เพิ่มชื่อคอลัมน์ที่ต้องการเปลี่ยนเพิ่มเติมที่นี่
})
# เลือกคอลัมน์ที่ต้องการ
selected_columns = ['File Name wav','label','tempo','rhythm','rest','num of pitch','File Name midi']
result_df = joined_df[selected_columns]
# บันทึกเป็นไฟล์ CSV ใหม่
result_df.to_csv(f'../data/exp{exp[0]}-{exp[1]}/{exp[0]}-{exp[1]}-{exp[2]}/file_names_MT_exp{exp[0]}-{exp[1]}-{exp[2]}_validate.csv', index=False)